# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  May 26 2022  2:04PM  238192        10     Eme-99959   
1  May 26 2022  1:56PM  238191        10    SNAP526221   
2  May 26 2022  1:48PM  238190        15  AL-000001085   
3  May 26 2022  1:47PM  238189        15       6507750   
4  May 26 2022  1:32PM  238185        10    0085714982   
5  May 26 2022  1:32PM  238185        10    0085715112   
6  May 26 2022  1:32PM  238185        10    0085715123   
7  May 26 2022  1:32PM  238185        10    0085715122   
8  May 26 2022  1:32PM  238185        10    0085715126   
9  May 26 2022  1:32PM  238185        10    0085715136   

                                Customer ShipmentStatus  
0                            Emerginnova       Released  
1           Snap Medical Industries, LLC       Released  
2                           Alexso, Inc.       Released  
3  Carwin Pharmaceutical Associates, LLC       Released  
4                      ISDIN Corporation       Released  
5                      ISDIN Corporation       Released  
6                      ISDIN Corporation       Released  
7                      ISDIN Corporation       Released  
8                      ISDIN Corporation       Released  
9                      ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  238186       Released          2
40  238189       Released          1
41  238190       Released          1
42  238191       Released          1
43  238192       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238186                NaN        NaN       2.0
238189                NaN        NaN       1.0
238190                NaN        NaN       1.0
238191                NaN        NaN       1.0
238192                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238100                0.0        0.0       1.0
238101                0.0        0.0      20.0
238105                0.0        0.0       1.0
238108                0.0        0.0       1.0
238117                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238100                  0          0         1
238101                  0          0        20
238105                  0          0         1
238108                  0          0         1
238117                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238100          0          0         1
1               238101          0          0        20
2               238105          0          0         1
3               238108          0          0         1
4               238117          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               238100                            1
1               238101                           20
2               238105                            1
3               238108                            1
4               238117                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   May 26 2022  2:04PM  238192        10   
1   May 26 2022  1:56PM  238191        10   
2   May 26 2022  1:48PM  238190        15   
3   May 26 2022  1:47PM  238189        15   
4   May 26 2022  1:32PM  238185        10   
24  May 26 2022  1:31PM  238186        10   
26  May 26 2022  1:13PM  238181        16   
27  May 26 2022  1:08PM  238179        10   
28  May 26 2022  1:05PM  238178        16   
29  May 26 2022  1:03PM  238177        10   

                                 Customer  
0                             Emerginnova  
1            Snap Medical Industries, LLC  
2                            Alexso, Inc.  
3   Carwin Pharmaceutical Associates, LLC  
4                       ISDIN Corporation  
24                      ISDIN Corporation  
26         Sartorius Bioprocess Solutions  
27          Nextsource Biotechnology, LLC  
28     Sartorius Lab Products and Service  
29                       Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  May 26 2022  2:04PM  238192        10   
1  May 26 2022  1:56PM  238191        10   
2  May 26 2022  1:48PM  238190        15   
3  May 26 2022  1:47PM  238189        15   
4  May 26 2022  1:32PM  238185        10   
5  May 26 2022  1:31PM  238186        10   
6  May 26 2022  1:13PM  238181        16   
7  May 26 2022  1:08PM  238179        10   
8  May 26 2022  1:05PM  238178        16   
9  May 26 2022  1:03PM  238177        10   

                                Customer Completed Executing Released  
0                            Emerginnova                            1  
1           Snap Medical Industries, LLC                            1  
2                           Alexso, Inc.                            1  
3  Carwin Pharmaceutical Associates, LLC                            1  
4                      ISDIN Corporation                           20  
5                      ISDIN Corporation                            2  
6         Sartorius Bioprocess Solutions                            1  
7          Nextsource Biotechnology, LLC                            1  
8     Sartorius Lab Products and Service                            1  
9                       Methapharm, Inc.                            7

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  May 26 2022  2:04PM  238192        10   
1  May 26 2022  1:56PM  238191        10   
2  May 26 2022  1:48PM  238190        15   
3  May 26 2022  1:47PM  238189        15   
4  May 26 2022  1:32PM  238185        10   
5  May 26 2022  1:31PM  238186        10   
6  May 26 2022  1:13PM  238181        16   
7  May 26 2022  1:08PM  238179        10   
8  May 26 2022  1:05PM  238178        16   
9  May 26 2022  1:03PM  238177        10   

                                Customer Released Executing Completed  
0                            Emerginnova        1                      
1           Snap Medical Industries, LLC        1                      
2                           Alexso, Inc.        1                      
3  Carwin Pharmaceutical Associates, LLC        1                      
4                      ISDIN Corporation       20                      
5                      ISDIN Corporation        2                      
6         Sartorius Bioprocess Solutions        1                      
7          Nextsource Biotechnology, LLC        1                      
8     Sartorius Lab Products and Service        1                      
9                       Methapharm, Inc.        7

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  May 26 2022  2:04PM  238192        10   
1  May 26 2022  1:56PM  238191        10   
2  May 26 2022  1:48PM  238190        15   
3  May 26 2022  1:47PM  238189        15   
4  May 26 2022  1:32PM  238185        10   

                                Customer Released Executing Completed  
0                            Emerginnova        1                      
1           Snap Medical Industries, LLC        1                      
2                           Alexso, Inc.        1                      
3  Carwin Pharmaceutical Associates, LLC        1                      
4                      ISDIN Corporation       20

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse  \
0  May 26 2022  2:04PM  238192        10   
1  May 26 2022  1:56PM  238191        10   
2  May 26 2022  1:48PM  238190        15   
3  May 26 2022  1:47PM  238189        15   
4  May 26 2022  1:32PM  238185        10   

                                Customer  Released  Executing  Completed  
0                            Emerginnova       1.0        NaN        NaN  
1           Snap Medical Industries, LLC       1.0        NaN        NaN  
2                           Alexso, Inc.       1.0        NaN        NaN  
3  Carwin Pharmaceutical Associates, LLC       1.0        NaN        NaN  
4                      ISDIN Corporation      20.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  May 26 2022  2:04PM  238192        10   
1  May 26 2022  1:56PM  238191        10   
2  May 26 2022  1:48PM  238190        15   
3  May 26 2022  1:47PM  238189        15   
4  May 26 2022  1:32PM  238185        10   

                                Customer  Released  Executing  Completed  
0                            Emerginnova       1.0        0.0        0.0  
1           Snap Medical Industries, LLC       1.0        0.0        0.0  
2                           Alexso, Inc.       1.0        0.0        0.0  
3  Carwin Pharmaceutical Associates, LLC       1.0        0.0        0.0  
4                      ISDIN Corporation      20.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3334370      71.0       27.0       23.0
15          714518       3.0       21.0        0.0
16         1190710       4.0        1.0        0.0
18          476292       4.0        8.0        0.0
19          476280       5.0        0.0        0.0
20         1667073      54.0        2.0        0.0
21          476284       2.0        0.0        0.0
22          476226       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3334370      71.0       27.0       23.0
1        15   714518       3.0       21.0        0.0
2        16  1190710       4.0        1.0        0.0
3        18   476292       4.0        8.0        0.0
4        19   476280       5.0        0.0        0.0
5        20  1667073      54.0        2.0        0.0
6        21   476284       2.0        0.0        0.0
7        22   476226       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      71.0       27.0       23.0
1        15       3.0       21.0        0.0
2        16       4.0        1.0        0.0
3        18       4.0        8.0        0.0
4        19       5.0        0.0        0.0
5        20      54.0        2.0        0.0
6        21       2.0        0.0        0.0
7        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   71.0
1        15  Released    3.0
2        16  Released    4.0
3        18  Released    4.0
4        19  Released    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   18   19    20   21   22
Status                                              
Completed  23.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0
Executing  27.0  21.0  1.0  8.0  0.0   2.0  0.0  0.0
Released   71.0   3.0  4.0  4.0  5.0  54.0  2.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   18   19    20   21   22
0          Completed  23.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0
1          Executing  27.0  21.0  1.0  8.0  0.0   2.0  0.0  0.0
2           Released  71.0   3.0  4.0  4.0  5.0  54.0  2.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   18   19    20   21   22
0  Completed  23.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  Executing  27.0  21.0  1.0  8.0  0.0   2.0  0.0  0.0
2   Released  71.0   3.0  4.0  4.0  5.0  54.0  2.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()